# Volume 3: MongoDB.
    Matthew Schaelling
    Math 403
    November 16, 2017

In [1]:
import re
import json
from pymongo import MongoClient

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [2]:
client = MongoClient()
db1 = client.db1
col1 = db1.collection2
col1.delete_many({})

with open('trump.json', 'r') as trump:
    lines = trump.readlines()
    for line in lines:
        col1.insert_one(json.loads(line))
col1.count()

95643

## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [3]:
russia = col1.find({'text': {'$regex':re.compile('russia', re.IGNORECASE)}})
us_timezones = col1.find({'user.time_zone':{'$in': ["Central Time (US & Canada)", "Pacific Time (US & Canada)", "Eastern Time (US & Canada)", "Mountain Time (US & Canada)"]}})
languages = col1.distinct('lang')
lang_freq = dict()
for language in languages:
    lang_freq[language] = col1.find({'lang':language}).count()

print('"Russia" occurs {} times.'.format(russia.count()))
print("{} tweets came from one of the main continental US time zones.".format(us_timezones.count()))
for language in languages:
    print(language + ':\t' + str(lang_freq[language]))

"Russia" occurs 5841 times.
33939 tweets came from one of the main continental US time zones.
en:	84919
es:	3027
fr:	1235
und:	1653
nl:	212
it:	423
pt:	632
de:	635
tl:	48
tr:	588
ru:	29
ro:	21
in:	121
pl:	108
cs:	13
ja:	17
et:	23
no:	21
lt:	8
sl:	3
th:	3
sv:	55
hu:	6
uk:	3
da:	35
eu:	6
ht:	21
el:	10
vi:	7
fi:	20
ar:	30
fa:	5
iw:	1
ko:	2
lv:	2
ur:	1
cy:	2
is:	1
zh:	1


## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [4]:
# Find most popular usernames
sorted_users = col1.find({}).sort('user.followers_count', -1)
popular_users = []
i = 0
while len(popular_users) < 5:
    if sorted_users[i]['user']['screen_name'] not in popular_users:
        popular_users.append(sorted_users[i]['user']['screen_name'])
    i += 1 
print(popular_users)

['CNN', 'Reuters', 'FoxNews', 'WSJ', 'TIME']


In [5]:
# Find tweets with 5 hashtags
five_hashtags = col1.find({'entities.hashtags.4': {'$exists': True}})
five_hashtags = five_hashtags.sort('entities.hashtags.4.indices.0')
print("The earliest character account it appears at is: {}".format(
    five_hashtags[0]['entities']['hashtags'][4]['indices'][0]+1))

The earliest character account it appears at is: 24


In [6]:
# 
sorted_coordinates = col1.find({'coordinates.coordinates':
                                {'$exists':
                                 True}}).sort('coordinates.coordinates.1', -1)
print("The coordinates of the furthest north one is: {}".format(
    sorted_coordinates[0]['coordinates']['coordinates']))

The coordinates of the furthest north one is: [19.4544773, 51.7765107]


## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.
- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [7]:
col1.update_many({},{"$unset": {'retweeted_status': ''}})
print("Number of Tweets with 'retweeted_status': {}".format(col1.find({'retweeted_status':{'$exists':True}}).count()))

col1.update_many({'user.followers_count': {'$gte': 1000, '$exists': True}}, 
                 {'$set': {'popular': True}})
print("Number of Popular Tweets: {}".format(col1.find({'popular': {"$exists":True}}).count()))

Number of Tweets with 'retweeted_status': 0
Number of Popular Tweets: 32489
